In [1]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

In [2]:
companies_with_cap = pd.read_csv('../data/companies_after_2005.csv', encoding='latin1')
svi = pd.read_csv('../data/weekly_svi.csv', encoding='latin1')
edgar = pd.read_csv('../data/edgar_all.csv', encoding='latin1')
weekly_market = pd.read_csv('../data/Weekly_market.csv', encoding='latin1')

#function to check wether there are equal number of weeks each year and companies 
#what should be in a dataset: company symbols, week number, week return,week volume, svi, edgar, cap size
#to see: returns from svi all, retturns from svi high, returns from svi low, returns from edgar all/high/all
#the same with volume (function)
#How to choose a lag

In [33]:
companies_with_cap.head(5)

,Unnamed: 0,Rank,Name,Symbol,marketcap,price (USD),country,IPO_year,capitalization
0,0,1,Microsoft,MSFT,3002343620608,404.06,United States,1986,high
1,1,2,Apple,AAPL,2815212650496,182.31,United States,1980,high
2,2,4,NVIDIA,NVDA,1793541144576,726.13,United States,1999,high
3,3,5,Amazon,AMZN,1760768032768,169.51,United States,1997,high
4,4,6,Alphabet (Google),GOOG,1753911394304,141.76,United States,2004,high


In [35]:
common_values = (
    set(weekly_market['TICKER'])
    .intersection(set(svi['symbol']))
    .intersection(set(companies_with_cap['Symbol']))
    .intersection(set(edgar['Symbol']))
)

# Filter each dataset to include only rows with common 'Symbol' values
filtered_weekly_market = weekly_market[weekly_market['TICKER'].isin(common_values)]
filtered_svi = svi[svi['symbol'].isin(common_values)]
filtered_companies = companies_with_cap[companies_with_cap['Symbol'].isin(common_values)]
filtered_edgar = edgar[edgar['Symbol'].isin(common_values)]

In [47]:
filtered_svi.shape

(325376, 5)

In [45]:
# Extract the year from 'week_index'
filtered_weekly_market['year'] = filtered_weekly_market['week_index'].str.split('-').str[0].astype(int)

# Group by 'year' and calculate the count for each group
weeks_per_year_df2 = filtered_weekly_market.groupby('year').size()

# Print the counts
print("Weeks per year in df2:")
print(weeks_per_year_df2)

Weeks per year in df2:
year
2005    14779
2006    15000
2007    15585
2008    15592
2009    15537
2010    15546
2011    15748
2012    16192
2013    16343
2014    16435
2015    16556
2016    16367
2017    16432
2018    16852
2019    16953
2020    17099
2021    16919
2022    16985
2023    17027
dtype: int64


C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1404412366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_weekly_market['year'] = filtered_weekly_market['week_index'].str.split('-').str[0].astype(int)


In [40]:
filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])

# Extract the year from the date
filtered_svi['year'] = filtered_svi['date'].dt.year

# Group by 'year' and calculate the count for each group
weeks_per_year = filtered_svi.groupby('year').size()

# Print the counts
print("Weeks per year:")
print(weeks_per_year)

Weeks per year:
year
2005    17056
2006    17384
2007    17056
2008    17056
2009    17056
2010    17056
2011    17056
2012    17384
2013    17056
2014    17056
2015    17056
2016    17056
2017    17384
2018    17056
2019    17056
2020    17056
2021    17056
2022    17056
2023    17384
dtype: int64


C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1774035501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['date'] = pd.to_datetime(filtered_svi['date'])
C:\Users\Варвара\AppData\Local\Temp\ipykernel_8116\1774035501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_svi['year'] = filtered_svi['date'].dt.year


In [24]:
filtered_df = weekly_market[weekly_market['week_index'].astype(str).str.contains('2005')]
filtered_df1 = svi[svi['date'].astype(str).str.contains('2005')]
filtered_df1.head(53)

,Unnamed: 0,symbol,date,svi
0,0,MSFT,2005-01-02,18
1,1,MSFT,2005-01-09,19
2,2,MSFT,2005-01-16,12
3,3,MSFT,2005-01-23,20
4,4,MSFT,2005-01-30,15
5,5,MSFT,2005-02-06,16
6,6,MSFT,2005-02-13,20
7,7,MSFT,2005-02-20,16
8,8,MSFT,2005-02-27,17
9,9,MSFT,2005-03-06,15


In [18]:
svi.head()

,Unnamed: 0,symbol,date,svi
0,0,MSFT,2005-01-02,18
1,1,MSFT,2005-01-09,19
2,2,MSFT,2005-01-16,12
3,3,MSFT,2005-01-23,20
4,4,MSFT,2005-01-30,15


In [25]:
weekly_market.head(5)

,TICKER,COMNAM,week_index,week_cls_prc,week_vol,weekly_return
0,ORCL,ORACLE CORP,2005-01,13.33,286434996.0,NaN
1,ORCL,ORACLE CORP,2005-02,13.63,260108378.0,0.022506
2,ORCL,ORACLE CORP,2005-03,13.31,199217982.0,-0.023478
3,ORCL,ORACLE CORP,2005-04,13.68,276016873.0,0.027799
4,ORCL,ORACLE CORP,2005-05,13.66,200942158.0,-0.001462


In [ ]:
#test function 
def grangertest(X, Y, lag_num, verbose=True):
    """
    Perform the Granger Causality Test on two time series.

    Parameters:
    - X: pandas Series or DataFrame
    - Y: pandas Series or DataFrame
    - lag_num: int, the number of lags to consider in the test
    - verbose: bool, whether to print the test results (default is True)

    Returns:
    - None
    """
    # Check if input data is a DataFrame, if not, convert to DataFrame
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=['X'])
    if not isinstance(Y, pd.DataFrame):
        Y = pd.DataFrame(Y, columns=['Y'])

    # Concatenate X and Y for the Granger Causality Test
    data = pd.concat([X, Y], axis=1)

    # Perform Granger Causality Test
    test_result = grangercausalitytests(data, lag_num, verbose=verbose)

    # Extract the p-values from the test results
    p_values = [test_result[i+1][0]['ssr_ftest'][1] for i in range(lag_num)]

    # Print the test statistics and conclusion
    if verbose:
        print(f"Granger Causality Test Results:")
        for i in range(1, lag_num + 1):
            print(f"\nLag {i} - p-value: {p_values[i-1]}")
            if p_values[i-1] < 0.05:
                print(f"Conclusion: There is Granger causality at lag {i}")
            else:
                print(f"Conclusion: There is no Granger causality at lag {i}")
